# Projet IC05 – Collecte et traitement des données des artistes issus des Top50

Ce script récupère les informations et fait des statistiques sur les artistes

## Imports et tokens

In [11]:
import requests
import csv
import pandas as pd
from credentials import ClientSecretCO as client_id, ClientIDCO as client_secret # TODO -  reparer ce import  

def get_acces_token(client_id, client_secret):
    """retourne un json (dict)"""
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": f"{client_id}",
        f"client_secret": f"{client_secret}"
    }

    response = requests.post(url, headers=headers, data=data)

    return response.json()

# recupérer un acces token
AccessToken = get_acces_token(ClientIDCO, ClientSecretCO)
# print(AccessToken['access_token'])


## Collecte des données sur les artistes

### Correspondance entre les artistes et les pays dans lesquels ils apparaissent

In [12]:
# correspondance entre artists_id et country
def extract_artist_country_mapping(src_file):
    artist_country_mapping = {}
    with open(src_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # ignorer l'en-tête
        for row in reader:
            artist_ids = row[2].strip('"[]').replace(" ", "").split(",")
            country = row[-1]  # colonne "country"
            for artist_id in artist_ids:
                artist_country_mapping[artist_id] = country
    return artist_country_mapping

### Formatage des données JSON pour l'écriture dans un fichier CSV et écriture

In [15]:
# ecriture des données json dans un csv
def json_to_csv(json_data, output_filename, artists_countries):
    # Préparer le fichier CSV
    with open(output_filename, mode='a+', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["artists", "followers", "genres", "name", "popularity", "country"])
        if file.tell() == 0:  # ecrire l'en-tête uniquement si le fichier est vide
            writer.writeheader()

        # Traiter chaque artiste dans la liste
        for artist in json_data["artists"]:
            # Extraire l'ID à partir de l'URL Spotify
            full_url = artist.get("external_urls", {}).get("spotify", "")
            artist_id = full_url.split("/")[-1] if full_url else ""

            # Préparer les informations pertinentes pour chaque artiste
            row = {
                "artists": artist_id,
                "followers": artist.get("followers", {}).get("total", 0),
                "genres": ", ".join(artist.get("genres", [])),
                "name": artist.get("name", ""),
                "popularity": artist.get("popularity", 0),
                "country": artists_countries[artist_id].pop()
            }
            writer.writerow(row)

### Collecter les données des artistes (artists,followers,genres,name,popularity,country)

In [17]:
# retrieve artists info (name, genres,popularity, followers)
def get_artists_info(AccessToken, src_file, dst_file) :
    artists_countries = {}
    # dictionnaire qui garde pour quel pays on note tel ou tel artiste

    artists_ids_string = ""
    artists_ids_list = ""
    result =[]

    # lecture du fichier CSV et recup les artistes_id
    with open(src_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        first_row = next(reader)
        for row in reader:
            # print(row[2]) # col artistes
            if "," in row[2] :
                # print(row[2].strip('"[]').replace(" ", ""))
                artist_id_str = row[2].strip('"[]').replace(" ", "") # string id1,id2,id3...etc
                artists_ids_string += artist_id_str
                
                for artist_id in artist_id_str.split(",") :  
                # pour ajouter dans le dictionnaire tout les artistes
                    if artist_id not in artists_countries.keys():
                        artists_countries[artist_id] = []
                    artists_countries[artist_id].append(row[-1])
            else :
                # print(row[2][1:23])
                artist_id_str = row[2][1:23]
                artists_ids_string += artist_id_str
                if artist_id_str not in artists_countries.keys():
                    artists_countries[artist_id_str] = []
                artists_countries[artist_id_str].append(row[-1])

            artists_ids_string +=","
            
    # print(artists_ids_string)
    artists_ids_list = artists_ids_string.split(",")[:-1]

    
    for i in range(0, len(artists_ids_list), 50 ):
        chunk = artists_ids_list[i:i + 50]  # 50 artistes
        # print(','.join(chunk), "\n")
        response = requests.get(
            url = "https://api.spotify.com/v1/artists?ids=" +','.join(chunk),
            headers = {
                "Authorization": 'Bearer ' + AccessToken['access_token']
            }
        )
        # print(response.json())
        # requête réussie
        if response.status_code == 200:
            json_to_csv(response.json(), dst_file, artists_countries)
        # erreur requete
        else:
            print(f"Erreur lors de la requête pour le lot {i // 50 + 1}: {response.status_code}")

    #return artists_ids_list, artists_ids_list
get_artists_info(AccessToken, "data/continent_features/Africa.csv", "data/artists/artists_africa.csv")
get_artists_info(AccessToken, "data/continent_features/Asia.csv", "data/artists/artists_asia.csv")
get_artists_info(AccessToken, "data/continent_features/Central_America.csv", "data/artists/artists_central_america.csv")
get_artists_info(AccessToken, "data/continent_features/South_America.csv", "data/artists/artists_south_america.csv")
get_artists_info(AccessToken, "data/continent_features/North_America.csv", "./data/artists/artists_north_america.csv")
get_artists_info(AccessToken, "data/continent_features/Europe.csv", "data/artists/artists_europe.csv")
get_artists_info(AccessToken, "data/continent_features/Oceania.csv", "data/artists/artists_oceania.csv")
get_artists_info(AccessToken, "data/top50_features_avec_USA.csv", "data/artists/artists.csv")

### Suppression des lignes en double dans les fichiers de donnéees des artistes

In [18]:
def delete_duplicate(file) :
    df = pd.read_csv(file)
    df_sans_doublons = df.drop_duplicates()
    df_sans_doublons.to_csv(file, index=False)

# delete_duplicate("data/artists/test-usa.csv")
# delete_duplicate("data/artists/artists_europe.csv")
# delete_duplicate("data/artists/artists_oceania.csv")
# delete_duplicate("data/artists/artists_north_america.csv")
# delete_duplicate("data/artists/artists_south_america.csv")
# delete_duplicate("data/artists/artists_central_america.csv")
# delete_duplicate("data/artists/artists_asia.csv")
# delete_duplicate("data/artists/artists_africa.csv")

## Statistiques sur les artistes

In [18]:
from collections import Counter, defaultdict

def statistiques_artistes(fichier="data/artists_info/apparitions_artistes.csv"):
    # stocker les informations
    artist_country_count = defaultdict(Counter)
    country_count = defaultdict(set)
    
    # lecture du fichier
    with open(fichier, 'r', encoding='utf-8') as file:
        next(file)  # ignorer premiere ligne
        for line in file:
            line = line.strip()
            # cas particulier "Tyler, The Creator"
            if line.startswith('"Tyler, The Creator"'):
                artist = '"Tyler, The Creator"'
                rest = line[len(artist):].strip(',')
                genre, country = rest.split(',')
            else:
                parts = line.split(',')
                artist, genre, country = parts
            
            # occurrences
            artist_country_count[artist][country] += 1
            country_count[artist].add(country)


    # 10 artistes les plus frequent avec leurs pays
    sorted_artists = sorted(
        artist_country_count.items(),
        key=lambda x: sum(x[1].values()), 
        reverse=True
    )[:10]
    top_10_artists = [
        (artist, dict(country_counts))
        for artist, country_counts in sorted_artists
    ]

    # 3 artistes avec le plus de pays differents
    diverse_artists = sorted(
        country_count.items(),
        key=lambda x: len(x[1]),
        reverse=True
    )[:10]
    top_3_diverse_artists = [
        (artist, len(countries))
        for artist, countries in diverse_artists
    ]

    return top_10_artists, top_3_diverse_artists

top_10, top_3 = statistiques_artistes("data/artists_info/apparitions_artistes.csv")

print("Top 10 des artistes les plus fréquents avec les pays dans lesquels ils sont écoutés:")
for artist, countries in top_10:
    print(f"{artist}: {countries}")

print("\nTop 10 des artistes peuplant le plus de Top50 différents:")
for artist, country_count in top_3:
    print(f"{artist}: {country_count} pays")


Top 10 des artistes les plus fréquents avec les pays dans lesquels ils sont écoutés:
Bruno Mars: {'United Kingdom': 4, 'Ukraine': 6, 'Switzerland': 9, 'Sweden': 4, 'Spain': 3, 'Slovakia': 4, 'Romania': 9, 'Poland': 6, 'Norway': 5, 'Netherlands': 6, 'Luxembourg': 5, 'Lithuania': 4, 'Latvia': 4, 'Italy': 14, 'Ireland': 8, 'Iceland': 8, 'Hungary': 6, 'Greece': 3, 'Germany': 9, 'France': 6, 'Finland': 5, 'Estonia': 10, 'Denmark': 4, 'Czech Republic': 4, 'Bulgaria': 7, 'Belgium': 6, 'Austria': 10, 'Andorra': 3, 'South Africa': 3, 'Morocco': 28, 'Mexico': 6, 'Canada': 6, 'United States': 16, 'Venezuela': 5, 'Peru': 6, 'Paraguay': 2, 'Ecuador': 6, 'Colombia': 5, 'Chile': 6, 'Brazil': 6, 'Bolivia': 4, 'Panama': 15, 'Nicaragua': 4, 'Honduras': 10, 'Guatemala': 5, 'El Salvador': 24, 'Dominican Republic': 2, 'Dominica': 5, 'Costa Rica': 4, 'Vietnam': 4, 'Turkey': 8, 'Thailand': 6, 'Taiwan': 9, 'Singapore': 13, 'Saudi Arabia': 4, 'Philippines': 12, 'Pakistan': 12, 'Malaysia': 11, 'Kazakhstan': 6, 